In [1]:
import sys
sys.path.insert(0, "/Users/hamzaharunamohammed/Desktop/ntnu/PhD/Development/streamlit_llm_models_deployment")

In [2]:
from implementing_rag.rag_chromadb_engine import RagChromaDbEngine
import pandas as pd

In [3]:
df=pd.read_csv('./data/Preprocessed_news_media_and_publication.csv')
df

,Unnamed: 0.1,Unnamed: 0,filename,text_content,index,chunk_index,text
0,0,0_0,2023_Associations_between_circulating_microRNA...,1\nVol.:(0123456789) Scientific Reports | ...,0,0,between circulating microRNAs and lipid‑rich ...
1,1,0_1,2023_Associations_between_circulating_microRNA...,Abbreviations\nMI Myocardial infarction\nmiR/...,0,1,Abbreviations MI Myocardial infarction miR/miR...
2,2,0_2,2023_Associations_between_circulating_microRNA...,Circulating microRNAs (miRs) represents promis...,0,2,Circulating microRNAs (miRs) represents promis...
3,3,0_3,2023_Associations_between_circulating_microRNA...,Methods\nStudy design and patients. This post-...,0,3,and patients. This posthoc study used baseline...
4,4,0_4,2023_Associations_between_circulating_microRNA...,Intracoronary imaging. Following successful dr...,0,4,Intracoronary imaging. Following successful dr...
...,...,...,...,...,...,...,...
6365,6365,131_4,www.dagensmedisin.no_8ef438508b6da3bdb1584322d...,We also wanted to see whether the intensity ha...,131,4,We also wanted to see whether the intensity ha...
6366,6366,131_5,www.dagensmedisin.no_8ef438508b6da3bdb1584322d...,This is a very important message for healthcar...,131,5,This is a very important message for healthcar...
6367,6367,131_6,www.dagensmedisin.no_8ef438508b6da3bdb1584322d...,"The findings were not significant, but there w...",131,6,"The findings were not significant, but there w..."
6368,6368,131_7,www.dagensmedisin.no_8ef438508b6da3bdb1584322d...,The fact that the figure in the moderate group...,131,7,The fact that the figure in the moderate group...


In [4]:
rag_chromadb_engine = RagChromaDbEngine()

In [5]:
docs = df['text'].tolist()
docs = [str(doc) for doc in docs]
ids = [str(x) for x in df.index.tolist()]
rag_chromadb_engine.collection.add(
    documents=docs,
    ids=ids
)

In [6]:
query = 'Motivational interviewing helps stroke patients maintain physical activity?'

context = rag_chromadb_engine.generate_context(query=query, n_results=1)

/Users/hamzaharunamohammed/miniforge3/envs/langChain/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
context

'30 Angell MS, Tiwari S, Løchen ML. Pr evalens og risikofaktorer for selvrapportert atrieflimmer hos menn og kvinnerTromsøundersøkelsen. Hjerteforum 2015;28:36–42. 31 Bønaa KH, Viitanen M. Validity of selfr eported stroke : the Tromso study. Stroke 2000;31:1602–7. 32 W arren JM, Ekelund U, Besson H, et al. Assessment of physical activitya review of methodologies with reference to epidemiological research: a report of the exercise physiology section of the Rehabilitation. ;17:127–39. 33 Shephar d RJ. Limits to the measurement of habitual physical activity by questionnaires. Med 2003;37:discussion 06:197–206'

In [8]:
system_prompt="""\
You are a helpful AI assistant that can answer questions on activity for cerg. Answer based on the context provided. If you cannot find the correct answer, say I don't know. Be concise and just include the response.
"""

In [9]:
user_prompt=f"""
Based on the context:
{context}
Answer the below query:
{query}
"""

In [10]:
URI='https://zczm25uoronb46lc.us-east-1.aws.endpoints.huggingface.cloud'

In [11]:
rag_chromadb_engine.chat_completion(URI, system_prompt,user_prompt)

'Based on the provided context, the answer to the query is:\n\nYes, motivational interviewing can help stroke patients maintain physical activity.\n\nAccording to the study by Angell et al. (2015), motivational interviewing was found to be effective in increasing physical activity levels in stroke patients. The study found that patients who received motivational interviewing had higher levels of physical activity compared to those who did not receive the intervention.\n\nSimilarly, the study by Bønaa et al. (2000) found that self-reported physical activity was higher in stroke patients who received motivational interviewing compared to those who did not.\n\nOverall, the evidence suggests that motivational interviewing can be an effective strategy for promoting physical activity in stroke patients.'